In [ ]:
import argparse
import os
import shutil
from src.analysis.project_parser import parse_project
from src.analysis.dependency_resolver import resolve_dependencies
from src.analysis.feature_mapper import map_features_to_components
from src.generation.project_generator import generate_project
from src.generation.feature_integration import integrate_features

from src.models.prompt_templates import get_prompt_template
from src.utils.logger import logger
from src.utils.file_operations import read_file, write_file
from src.vector_database.db_builder import build_vector_database
from src.vector_database.db_query import query_vector_database


In [1]:
from src.vector_database.db_load import load_vector_db
from src.models.llm_inference import query_llm
prompt = "How to integrate the specified features into the project components?"
# Define the persistent directory and collection name
persist_directory = "./notebooks/chroma_db"
collection_name = "my_vector_collection"
# Load the vector database
vector_db = load_vector_db(collection_name, persist_directory)
# Query the vector database using the `query_llm` function
response = query_llm(prompt,vector_db)
# Print the response
print("Generated Response:")
print(response)

/mnt/c/Blog/Building-LLM-from-Scratch-in-Python/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/mnt/c/Blog/FactoryFeature/src/vector_database/db_load.py:38: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  return Chroma(collection_name=collection_name, embedding_function=hf, client=chroma_client)
/mnt/c/Blog/Building-LLM-from-Scratch-in-Python/.venv/lib/python3.10/site-packages/ibm_watsonx_ai/foundation_models/utils/utils.py:415: LifecycleWarning: Model 'meta-llama/llama-3-70b-instruct' is in deprecated state f

Generated Response:
{'query': 'How to integrate the specified features into the project components?', 'result': ' You can integrate the specified features into the project components by using the `integrate_features` function from the `feature_integration.py` file. This function takes two arguments: `project_components` and `feature_instructions`. It modifies the project components according to the feature instructions.'}


In [ ]:
prompt = "Who is man's best friend?"
response = query_llm(prompt)
# Print the response
print("Generated Response:")
print(response)

In [9]:
prompt="Add a user authentication feature to the project"

In [6]:
logger.info("Starting the Factory Feature program")
# Define paths
old_project_path = "project_old"
new_project_path = "project_new"

In [7]:
# Step 1: Parse the old project
logger.info("Parsing the project structure")
project_data = parse_project(old_project_path)
vector_db = build_vector_database(project_data)
logger.info("Vector database created for the project")

/mnt/c/Blog/FactoryFeature/src/vector_database/db_builder.py:16: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")


In [8]:
# Step 2: Resolve dependencies
logger.info("Resolving project dependencies")
dependencies = resolve_dependencies(old_project_path)
logger.info(f"Dependencies found: {dependencies.keys()}")

In [9]:
# Step 3: Map features to project components
logger.info("Mapping feature request to project components")
feature_template = get_prompt_template()
project_context = "\n".join([f"{key}: {value}" for key, value in dependencies.items()])
feature_request = feature_template.format(feature_request=prompt, project_context=project_context)
relevant_docs = query_vector_database(vector_db, feature_request)
logger.info(f"Relevant documents for the feature request: {[doc.metadata['source'] for doc in relevant_docs]}")

/mnt/c/Blog/FactoryFeature/src/vector_database/db_query.py:14: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  return retriever.get_relevant_documents(query)
Number of requested results 5 is greater than number of elements in index 3, updating n_results = 3


In [10]:
# Step 4: Generate new project structure
logger.info("Generating the updated project structure")
if os.path.exists(new_project_path):
    shutil.rmtree(new_project_path)
shutil.copytree(old_project_path, new_project_path)

'project_new'

In [11]:
# Step 5: Integrate new features
logger.info("Integrating new features into the project")
for doc in relevant_docs:
    integrate_features([doc.metadata["source"]], f"# Feature: {prompt}\n{doc.page_content}")

In [12]:
# Step 6: Generate additional files or modify existing ones
feature_instructions = [
    {"file": os.path.relpath(doc.metadata["source"], old_project_path), "content": f"# Feature: {prompt}\n{doc.page_content}"}
    for doc in relevant_docs
]

In [13]:
feature_instructions

[{'file': 'requirements.txt',
  'content': '# Feature: How to integrate the specified features into the project components?\nPath: project_old/requirements.txt\nContent:\n# Python dependencies\nflask\n\n# Feature Integration\n# Feature: Add a user authentication feature to the project\nPath: project_old/requirements.txt\nContent:\n# Python dependencies\nflask\n\n# Feature Integration\n# Feature: Add a user authentication feature to the project\nPath: project_old/requirements.txt\nContent:\n# Python dependencies\nflask\n\n# Feature Integration\n# Feature: Add a user authentication feature to the project\nPath: project_old/requirements.txt\nContent:\n# Python dependencies\nflask\n'},
 {'file': 'app.py',
  'content': '# Feature: How to integrate the specified features into the project components?\nPath: project_old/app.py\nContent:\nfrom utils.helpers import greet\n\ndef main():\n    name = input("Enter your name: ")\n    print(greet(name))\n\nif __name__ == "__main__":\n    main()\n\n# F

In [14]:
generate_project(old_project_path, new_project_path, feature_instructions)
logger.info("Feature integration completed")
print(f"Project updated with the feature: {prompt}")
print(f"Updated project saved in: {new_project_path}")

Project updated with the feature: How to integrate the specified features into the project components?
Updated project saved in: project_new
